<a href="https://colab.research.google.com/github/jiissung/ECON3916-Statistical-Machine-Learning/blob/main/Class%209/Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

df = pd.read_csv('lalonde.csv')

In [7]:
treated_mean = df[df['treat'] == 1]['re78'].mean()
control_mean = df[df['treat'] == 0]['re78'].mean()

# Naive Comparison
naive_diff = treated_mean - control_mean
print(f"Naive Difference in Means: ${naive_diff:,.2f}")
# Expected Result: -$635.03

naive_diff = treated_mean - control_mean

print(df)

Naive Difference in Means: $-635.03
     Unnamed: 0  treat  age  educ  black  hispan  married  nodegree  re74  \
0             1      1   37    11      1       0        1         1   0.0   
1             2      1   22     9      0       1        0         1   0.0   
2             3      1   30    12      1       0        0         0   0.0   
3             4      1   27    11      1       0        0         1   0.0   
4             5      1   33     8      1       0        0         1   0.0   
..          ...    ...  ...   ...    ...     ...      ...       ...   ...   
609         610      0   18    11      0       0        0         1   0.0   
610         611      0   24     1      0       1        1         1   0.0   
611         612      0   21    18      0       0        0         0   0.0   
612         613      0   32     5      1       0        1         1   0.0   
613         614      0   16     9      0       0        0         1   0.0   

     re75        re78  
0     0.0   993

In [17]:
# Define covariates
covariates = ['age','educ','black','hispan','married','nodegree','re74','re75']

X = df[covariates]
y = df['treat']

# Fit Propensity Model
logit = LogisticRegression(solver='liblinear')
logit.fit(X, y)

# Generate Scores
df['pscore'] = logit.predict_proba(X)[:, 1]


print(df['pscore'])

0      0.443350
1      0.144660
2      0.722355
3      0.664151
4      0.698286
         ...   
609    0.136895
610    0.108708
611    0.123129
612    0.550460
613    0.155030
Name: pscore, Length: 614, dtype: float64


In [13]:
from sklearn.neighbors import NearestNeighbors

treated = df[df.treat==1]
control = df[df.treat==0]


nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nbrs.fit(control[['pscore']])

# Find matches for Treated scores
distances, indices = nbrs.kneighbors(treated[['pscore']])
matched_control = control.iloc[indices.flatten()]

# Construct Matched DataFrame
matched_df = pd.concat([treated, matched_control])

In [22]:
from scipy import stats

def smd(x_t, x_c):
    return (x_t.mean() - x_c.mean()) / np.sqrt((x_t.var() + x_c.var()) / 2)

print("Standardized Mean Differences (After Matching):\n")

for cov in covariates:
    smd_value = smd(
        matched_df[matched_df.treat==1][cov],
        matched_df[matched_df.treat==0][cov]
    )
    print(f"{cov}: {abs(smd_value):.3f}")

# T-test on raw data
diff = treated['re78'].mean() - control['re78'].mean()
t_stat, p_val = stats.ttest_ind(treated['re78'], control['re78'])

print(f"Raw Effect (Difference): ${diff:,.2f}")
print(f"P-value: {p_val:.4f}")


# Isolate the matched outcomes
matched_treated = matched_df[matched_df.treat==1]['re78']
matched_control = matched_df[matched_df.treat==0]['re78']

# Estimate the causal effect (T-test on matched data)
matched_diff = matched_treated.mean() - matched_control.mean()
t_stat, p_val = stats.ttest_ind(matched_treated, matched_control)


print(f"Recovered Effect (Matched Difference): ${matched_diff:,.2f}")
print(f"P-value: {p_val:.4f}")

Standardized Mean Differences (After Matching):

age: 0.261
educ: 0.027
black: 0.015
hispan: 0.129
married: 0.131
nodegree: 0.206
re74: 0.049
re75: 0.080
Raw Effect (Difference): $-635.03
P-value: 0.3342
Recovered Effect (Matched Difference): $583.04
P-value: 0.4438
